In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Layer, UpSampling2D, MaxPooling2D
from keras.layers import Conv2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from osgeo import gdal

%matplotlib inline

Using TensorFlow backend.


In [2]:
img_rows, img_cols, channels = 64, 64, 2
num_conv = 3
batch_size = 16
original_img_size = (img_rows, img_cols, channels)
interm_dim = 128
latent_dim = 2
epsilon_std = 1.0
c_fil_n = [512, 512, 512, 512, 512, 512]

x = Input(batch_shape=(batch_size,) + original_img_size)

conv_1 = Conv2D(c_fil_n[0], 3, padding='same', activation='relu')(x)
maxp_1 = MaxPooling2D((2, 2), padding='same')(conv_1)#32

conv_2 = Conv2D(c_fil_n[1], 3, padding='same', activation='relu')(maxp_1)
maxp_2 = MaxPooling2D((2, 2), padding='same')(conv_2)#16

conv_3 = Conv2D(c_fil_n[2], 3, padding='same', activation='relu')(maxp_2)
maxp_3 = MaxPooling2D((2, 2), padding='same')(conv_3)#8

conv_4 = Conv2D(c_fil_n[3], 3, padding='same', activation='relu')(maxp_3)
maxp_4 = MaxPooling2D((2, 2), padding='same')(conv_4)#4

conv_5 = Conv2D(c_fil_n[4], 3, padding='same', activation='relu')(maxp_4)
maxp_5 = MaxPooling2D((2, 2), padding='same')(conv_5)#2

conv_6 = Conv2D(c_fil_n[5], 3, padding='same', activation='relu')(maxp_5)
maxp_6 = MaxPooling2D((2, 2), padding='same')(conv_6)#1


flat = Flatten()(maxp_6)

hidden = Dense(interm_dim, activation='relu')(flat)

z_mean = Dense(latent_dim)(hidden)
z_log_var = Dense(latent_dim)(hidden)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon


z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

decoder_hid = Dense(interm_dim, activation='relu')#(z)
decoder_upsample = Dense(c_fil_n[5] * 1 * 1, activation='relu')#(hid_decoded)

output_shape = (batch_size, 1, 1, c_fil_n[5])

decoder_reshape = Reshape(output_shape[1:])#(up_decoded)
up_0 = UpSampling2D((2,2))#(decoder_reshaped)#2
deconv_0 = Conv2D(c_fil_n[4], 3, padding='same', activation='relu')#(up_0)
up_05 = UpSampling2D((2, 2))#(deconv_0)#4
deconv_1 = Conv2D(c_fil_n[3], 3, padding='same', activation='relu')#(up_05)
up_1 = UpSampling2D((2, 2))#(deconv_1)#8
deconv_2 = Conv2D(c_fil_n[2], 3, padding='same', activation='relu')#(up_1)
up_2 = UpSampling2D((2, 2))#(deconv_2)#16
deconv_3 = Conv2D(c_fil_n[1], 3, padding='same', activation='relu')#(up_2)
up_3 = UpSampling2D((2, 2))#(deconv_3)#32
deconv_4 = Conv2D(c_fil_n[0], 3, padding='same', activation='relu')#(up_3)
up_4 = UpSampling2D((2, 2))#(deconv_4)#64
decoder_mean_squash = Conv2D(channels, 3, padding='same', activation='sigmoid')#(up_4)


hid_decoded = decoder_hid(z)
up_decoded = decoder_upsample(hid_decoded)
reshape_decoded = decoder_reshape(up_decoded)
up_0_decoded = up_0(reshape_decoded)
deconv_0_decoded = deconv_0(up_0_decoded)

up_05_decoded = up_05(deconv_0_decoded)
deconv_1_decoded = deconv_1(up_05_decoded)
up_1_decoded = up_1(deconv_1_decoded)
deconv_2_decoded = deconv_2(up_1_decoded)
up_2_decoded = up_2(deconv_2_decoded)
deconv_3_decoded = deconv_3(up_2_decoded)
up_3_decoded = up_3(deconv_3_decoded)
deconv_4_decoded = deconv_4(up_3_decoded)
up_4_decoded = up_4(deconv_4_decoded)
x_decoded_mean_squash = decoder_mean_squash(up_4_decoded)

# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean_squash):
        x = K.flatten(x)
        x_decoded_mean_squash = K.flatten(x_decoded_mean_squash)
        xent_loss = img_rows * img_cols * metrics.binary_crossentropy(x, x_decoded_mean_squash)
        kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean_squash = inputs[1]
        loss = self.vae_loss(x, x_decoded_mean_squash)
        self.add_loss(loss, inputs=inputs)
        # We don't use this output.
        return x

y = CustomVariationalLayer()([x, x_decoded_mean_squash])
vae = Model(x, y)
vae2 = Model(x, x_decoded_mean_squash)
vae.compile(optimizer='rmsprop', lr=0.001, loss=None)#lr=0.001 default
vae.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (16, 64, 64, 2)       0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (16, 64, 64, 512)     9728        input_1[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (16, 32, 32, 512)     0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (16, 32, 32, 512)     2359808     max_pooling2d_1[0][0]            
___________________________________________________________________________________________

/home/ron/.conda/envs/test1/lib/python2.7/site-packages/ipykernel_launcher.py:109: UserWarning: Output "custom_variational_layer_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "custom_variational_layer_1" during training.


In [3]:
def sample_a_batch(filebase, batchsize, tilesize=128, sb=False):
    # can sample for spatio-temporal (single_file = False), and spatial-only case (single_file = True).
    import numpy as np
    from osgeo import gdal
    
    if '.' in filebase:
        single_file = True
        S = gdal.Open(filebase)
    else:
        single_file = False
        S = gdal.Open(filebase + '_1.vrt')
        
    samples = []
    
    if single_file:
        while len(samples) < batchsize:
            RX = np.random.randint(S.RasterXSize-tilesize,size=1)
            RY = np.random.randint(S.RasterYSize-tilesize,size=1)
            
            A = np.transpose(S.ReadAsArray(RX[0], RY[0], tilesize, tilesize))
            if np.min(A) > 0.0:
                if sb:
                    samples.append(np.expand_dims(A[:, :, 1], 3))
                else:
                    samples.append(A)
        
    else:
        while len(samples) < batchsize:
            RX = np.random.randint(S.RasterXSize-tilesize,size=1)
            RY = np.random.randint(S.RasterYSize-tilesize,size=1) 
            
            skip_loc = False
            months = []
            
            for m in range(1,13):
                S = gdal.Open(filebase + '_' + str(m) + '.vrt')
                A = np.transpose(S.ReadAsArray(RX[i], RY[i], tilesize, tilesize))
                if np.min(A) == 0.0:
                    skip_loc = True
                    break
                else:
                    months.append(A)                
            if not skip_loc:
                months = np.array(months)
                samples.append(months)
        
    return np.array(samples)

In [4]:
def normalization_parameters(fn):
    S = gdal.Open(fn)
    mns = []
    sds = []
    maxs = []
    
    for b in range(S.RasterCount):
        B = S.GetRasterBand(b+1)
        mn, sd = B.ComputeStatistics(1)[2:4]
        mns.append(mn)
        sds.append(sd)
        maxs.append(B.GetMaximum())
        
    return mns, sds, maxs

In [6]:
filename = '/run/media/ron/silver_small/twelve_months/3d/S1A_IW_GRDH_1SDV_20160325T083601_20160325T083630_010523_00FA23_6F51.tif'
stats = normalization_parameters(filename)


In [ ]:
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#              patience=5, min_lr=0.001)
#model.fit(X_train, Y_train, callbacks=[reduce_lr])
#reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.000001, verbose=1, mode=min)

for i in range(4000):
    print i
    x_train = sample_a_batch(filename, 64*30, 64, sb=False)
    x_test = sample_a_batch(filename, 64*10, 64, sb=False)
    
    for b in range(2):
        x_train[:,:,:,b] = x_train[:,:,:,b] / stats[2][b]
        x_test[:,:,:,b] = x_test[:,:,:,b] / stats[2][b]
#        x_train[:, :, :, b] = (x_train[:, :, :, b] - stats[0][b]) / stats[1][b]
#        x_test[:, :, :, b] = (x_test[:, :, :, b] - stats[0][b]) / stats[1][b]       
        
    vae.fit(x_train,
        shuffle=True,
        epochs=300,
        batch_size=batch_size,
        validation_data=(x_test, x_test))#,
#        callbacks=[reduce_lr])
#    
#    vae.fit_generator(datagen,
#        shuffle=True,
#        epochs=300,
#        batch_size=32,
#        validation_data=(x_test, x_test))

0


/home/ron/.conda/envs/test1/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:2250: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 1920 samples, validate on 640 samples
Epoch 1/300
1920/1920 [==============================] - 70s - loss: 265318477974208576.0000 - val_loss: 304.5987
Epoch 2/300
1920/1920 [==============================] - 58s - loss: 299.3941 - val_loss: 302.0629
Epoch 3/300
1920/1920 [==============================] - 58s - loss: 293.4423 - val_loss: 291.3308
Epoch 4/300
1920/1920 [==============================] - 58s - loss: 292.0728 - val_loss: 288.4126
Epoch 5/300
1920/1920 [==============================] - 58s - loss: 291.3899 - val_loss: 288.7999
Epoch 6/300
1920/1920 [==============================] - 58s - loss: 291.0850 - val_loss: 288.7780
Epoch 7/300
1920/1920 [==============================] - 58s - loss: 290.5521 - val_loss: 288.1045
Epoch 8/300
1920/1920 [==============================] - 58s - loss: 290.6377 - val_loss: 286.5100
Epoch 9/300
1920/1920 [==============================] - 58s - loss: 290.2498 - val_loss: 288.8387
Epoch 10/300
1920/1920 [=======================

In [ ]:
n_rows = 4
n_cols = 6

vistis = x_train

fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(16, 12))
plt.setp(axes.flat, xticks=[], yticks=[])

reconstructed = vae2.predict(vistis, batch_size=batch_size)

s = np.random.randint(150)
i = s
for col in axes.T:
    vv = vistis[i,:,:,1]
    vh = vistis[i,:,:,0]
    vv_rec = reconstructed[i,:,:,1]
    vh_rec = reconstructed[i,:,:,0]
    
    col[0].imshow(vv)
    col[1].imshow(vv_rec)#, clim=(0.0, np.max(vv)))
    col[2].imshow(vh)
    col[3].imshow(vh_rec)#, clim=(0.0, np.max(vh)))
    
    i += 1


#i = 0
#for p in predicted[s:][:n_cols]:
#    axes.T[i][1].set_title(str(np.argmax(p)))
#    axes.T[i][3].set_title(str(np.argmax(p)))
#    i += 1
    
fig.subplots_adjust(bottom=0.05, right=0.95)
plt.show()

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=180,
    vertical_flip=True,
    horizontal_flip=True)

train_generator = datagen.flow(
        x_train,  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

In [ ]:
# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

# display a 2D plot of the digit classes in the latent space
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1])#, c=y_test)
#plt.colorbar()
plt.show()

In [ ]:
# build a digit generator that can sample from the learned distribution
#decoder_input = Input(shape=(latent_dim,))
#_hid_decoded = decoder_hid(decoder_input)
#_up_decoded = decoder_upsample(_hid_decoded)
#_reshape_decoded = decoder_reshape(_up_decoded)
#_deconv_1_decoded = decoder_deconv_1(_reshape_decoded)
#_deconv_2_decoded = decoder_deconv_2(_deconv_1_decoded)
#_x_decoded_relu = decoder_deconv_3_upsamp(_deconv_2_decoded)
#_x_decoded_mean_squash = decoder_mean_squash(_x_decoded_relu)
#generator = Model(decoder_input, _x_decoded_mean_squash)
decoder_input = Input(shape=(latent_dim,))
_hid_decoded = decoder_hid(decoder_input)
_up_decoded = decoder_upsample(_hid_decoded)
_reshape_decoded = decoder_reshape(_up_decoded)
_up_0_decoded = up_0(_reshape_decoded)
_deconv_0_decoded = deconv_0(_up_0_decoded)
_up_05_decoded = up_05(_deconv_0_decoded)
_deconv_1_decoded = deconv_1(_up_05_decoded)
_up_1_decoded = up_1(_deconv_1_decoded)
_deconv_2_decoded = deconv_2(_up_1_decoded)
_up_2_decoded = up_2(_deconv_2_decoded)
_deconv_3_decoded = deconv_3(_up_2_decoded)
_up_3_decoded = up_3(_deconv_3_decoded)
_deconv_4_decoded = deconv_4(_up_3_decoded)
_up_4_decoded = up_4(_deconv_4_decoded)
_x_decoded_mean_squash = decoder_mean_squash(_up_4_decoded)
generator = Model(decoder_input, _x_decoded_mean_squash)

# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 64
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = generator.predict(z_sample, batch_size=batch_size)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()


In [ ]:
import pickle
pickle.dump(vae, open( "vae_9hour_rms_0.001.p", "wb" ) )

In [ ]:
help